In [1]:
import os
import openai
import json

In [2]:
# Set up your OpenAI API key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(f"OPENAI_API_KEY Active: ({bool(OPENAI_API_KEY):})")
if not OPENAI_API_KEY:
   raise Exception("No variable for \"OPENAI_API_KEY\"")

# Set up your OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")

OPENAI_API_KEY Active: (True)


In [3]:
from openai import OpenAI

# Initialize the OpenAI client
client = OpenAI()


In [4]:
from pydantic import BaseModel
from typing import List, Optional
from enum import Enum

In [5]:
from backend.app import enums

# Define an enum for the correct option
class CorrectOption(str, Enum):
    option1 = '1'
    option2 = '2'
    option3 = '3'
    option4 = '4'

class BibleQuizQuestion(BaseModel):
    # section_name: str
    question_text: str
    option1: str
    option2: str
    option3: str
    option4: str
    correct_option: CorrectOption  # Ensures the correct option is a string between '1' and '4'
    bible_reference: str
    difficulty: enums.Difficulty # Uses the Difficulty Enum
    topic: enums.Topics
    tags: List[enums.Tag] = None
    hint: str

class BibleQuiz(BaseModel):
    section_name: str
    questions: List[BibleQuizQuestion]


class QAGenerator:
    
    def __init__(self, client, model:str="gpt-4o-2024-08-06"):
        self.client = client
        self.model = model
    
    def _system_prompt(self):
    
        # Construct the system prompt
        system_prompt = f"""
You are an AI specialized in generating Bible-based quiz questions and answers. Your purpose is to create quiz questions from specific chapters, verses, or topics from the Bible. 

Your output should contain 10+ multiple-choice questions that are factually accurate and based on the text. Each question should include:
- The question itself
- Four possible answer choices
- The correct answer choice (identified by an index)
- A reference to the specific Bible passage where the answer is found
- A difficulty level (beginner, intermediate, advanced)
- A topic or theme of the question
- A helpful hint related to the question if possible

All questions, answers, and references should be based on the King James Version (KJV) of the Bible. Ensure that the wording, names, and details of each question are consistent with how they are presented in the KJV, as other versions might use different terminology or phrasing. Focus on keeping the content aligned with what is discussed or referenced in the KJV.

The questions should be strictly derived from the following Bible passages:
- Genesis chapters 37-50
- Zechariah chapters 1-6
- Romans chapters 1-10


Ensure that the questions are clear, concise, and provide educational value to the quiz-taker.
"""
    
        return system_prompt
    
    def _user_prompt_text(self, topic, type_of_question = None):
        
        if type_of_question == None:
            user_prompt = f"""Based on the provided Bible topic "{topic}", generate multiple-choice questions. Ensure that each question has four options, with one correct answer clearly identified. Include any relevant context, Bible references, difficulty level, topic, and a hint to aid understanding.
        """
        else:
            user_prompt = f"""Based on the provided Bible topic "{topic}", generate {type_of_question} multiple-choice type questions. Ensure that each question has four options, with one correct answer clearly identified. Include any relevant context, Bible references, difficulty level, topic, and a hint to aid understanding.
        You need to generate {type_of_question} type questions based on "{topic}".
        """
            
        return user_prompt
    

    def generate_questions_text(self, topic, type_of_question=None):
        # Build the system and user prompts
        system_prompt = self._system_prompt()
        user_prompt = self._user_prompt_text(topic=topic, type_of_question=type_of_question)

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
        
        # Call the API and handle the response
        response = self.client.beta.chat.completions.parse(
            model=self.model,
            messages=messages,
            response_format=BibleQuiz
        )
        
        # Step 4: Process the response from the API
        return self._process_response(response)

    
    def _process_response(self, response):
        """Helper to process the response from the API."""
        qa_pairs_message = response.choices[0].message

        # Handle refusal case
        if qa_pairs_message.refusal:
            print(qa_pairs_message.refusal)
            return None
        
        # Parse and return the actual content
        message_content = qa_pairs_message.content
        json_response = json.loads(message_content)
        
        return json_response

In [6]:
# Instantiate QAGenerator
qa_generator = QAGenerator(client=client)

In [7]:
# # Define the topic for which you want to generate questions
# topic = enums.Topics.joseph_rise_to_power  # This should match the enums.Topics

# try:
#     generated_quiz = qa_generator.generate_questions_text(topic)
# except Exception as e:
#     print(f"An error occurred: {e}")
    
# generated_quiz.get("questions")

# Iterate

# TOPICS

In [8]:
import os
import json
from tqdm import tqdm  # Import tqdm for progress bar

# Directory to save data
data_dir = "data"

# Create directory if it does not exist
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

# Define the type of questions to be generated
types_of_questions = "Event Sequencing"

# Iterate through all topics in enums.Topics and generate quiz questions
for topic in tqdm(enums.Topics, desc="Generating Questions"):
    topic_str = topic.value  # Get the string value of the enum
    print(f"Generating questions for topic: {topic_str}")

    try:
        # Generate questions for the current topic string, passing type_of_question
        generated_quiz = qa_generator.generate_questions_text(topic_str, type_of_question=types_of_questions)
        
        # Check if there are generated questions
        if generated_quiz:
            # Create a safe filename by replacing spaces with underscores and converting to lowercase
            if types_of_questions is None:
                filename = f"{topic_str.replace(' ', '_').lower()}.json"
            else:
                filename = f"{topic_str.replace(' ', '_').lower()}_{types_of_questions.replace(' ', '_').lower()}.json"
            file_path = os.path.join(data_dir, filename)
            
            # Save the generated quiz to a JSON file
            with open(file_path, 'w') as f:
                json.dump(generated_quiz, f, indent=4)
            
            print(f"Saved {len(generated_quiz['questions'])} questions for topic '{topic_str}' to {file_path}")
        
        else:
            print(f"No questions generated for topic: {topic_str}")
        
    except Exception as e:
        print(f"An error occurred while generating questions for topic {topic_str}: {e}")


Generating Questions:   0%|          | 0/28 [00:00<?, ?it/s]

Generating questions for topic: Joseph's Story


Generating Questions:   4%|▎         | 1/28 [00:12<05:42, 12.68s/it]

Saved 10 questions for topic 'Joseph's Story' to data\joseph's_story_event_sequencing.json
Generating questions for topic: Joseph's Rise to Power


Generating Questions:   7%|▋         | 2/28 [00:20<04:09,  9.59s/it]

...

Saved 3 questions for topic 'God's Wrath and Mercy' to data\god's_wrath_and_mercy_event_sequencing.json
Generating questions for topic: God’s Faithfulness to Israel


Generating Questions:  96%|█████████▋| 27/28 [03:31<00:06,  6.68s/it]

Saved 1 questions for topic 'God’s Faithfulness to Israel' to data\god’s_faithfulness_to_israel_event_sequencing.json
Generating questions for topic: Adam and Christ Contrast


Generating Questions: 100%|██████████| 28/28 [03:41<00:00,  7.90s/it]

Saved 5 questions for topic 'Adam and Christ Contrast' to data\adam_and_christ_contrast_event_sequencing.json


# Chapters

In [9]:
# import os
# import json
# from tqdm import tqdm  # Import tqdm for progress bar

# # Directory to save data
# data_dir = "data"

# # Create directory if it does not exist
# if not os.path.exists(data_dir):
#     os.makedirs(data_dir)

# # Define the books and their respective chapters
# bible_books = {
#     "Genesis": range(37, 51),        # Chapters 37-50
#     "Zechariah": range(1, 7),        # Chapters 1-6
#     "Romans": range(1, 11)           # Chapters 1-10
# }


# types_of_questions = "Thematic Questions"
# # Thematic Questions
# # Fill in the blank - DONE
# # Character Identification


# # Iterate through all books and their chapters
# for book, chapters in bible_books.items():
#     for chapter in tqdm(chapters, desc=f"Generating Questions for {book}"):
#         book_chapter_str = f"{book} {chapter}"  # Create a string representation for the book and chapter
#         print(f"Generating questions for {book_chapter_str}")

#         try:
#             # Generate questions for the current book and chapter string
#             generated_quiz = qa_generator.generate_questions_text(book_chapter_str, type_of_question=types_of_questions)
            
#             # Check if there are generated questions
#             if generated_quiz:
#                 # Create a safe filename by replacing spaces with underscores and converting to lowercase
#                 if types_of_questions ==  None:
#                     filename = f"{book.lower()}_{chapter}.json"
#                 else:
#                     filename = f"{book.lower()}_{chapter} {types_of_questions}.json"
#                 file_path = os.path.join(data_dir, filename)
                
#                 # Save the generated quiz to a JSON file
#                 with open(file_path, 'w') as f:
#                     json.dump(generated_quiz, f, indent=4)
                
#                 print(f"Saved {len(generated_quiz['questions'])} questions for {book_chapter_str} to {file_path}")
            
#             else:
#                 print(f"No questions generated for {book_chapter_str}")
        
#         except Exception as e:
#             print(f"An error occurred while generating questions for {book_chapter_str}: {e}")